# BiseNet (Bilateral Segmentation Network for Real-time Semantic Segmentation)  
- 논문 리뷰

## Abstract  
- 요즘 Semantic Segmentation의 접근법은 실시간으로 처리하는 대신 성능을 포기하는 경우가 많다  
  
  
- 본 논문에서는 이러한 딜레마를 해결하기 위해 BiseNet을 제안한다  
    - **Spatial Path**
        - 공간적 정보를 제공하고 고해상도의 feature를 생성하기 위해 작은 stride로 수행    
    - **Context Path**  
        - 빠른 down sampling과 함께 충분한 receptive fields를 얻기 위해 수행  
          
- 위의 두 Path 외에도 FFM(Feature Fusion Module)를 소개한다  
- Input size는 $2048\times1024$이며 105 FPS의 빠른 속도와 68.4%의 높은 정확도를 가진다

## Introduction  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiseNet/figure/figure1.JPG?raw=true">

- 자율 주행 등에서 빠른 반응을 위해 효율적인 inference 속도를 필요로 함  
  
  
- 최근 real-time semantic segmentation 모델들은 inference를 가속화하는 방법으로 크게 3가지가 있다  
    - 1. 이미지를 crop or resize를 진행  
        - 간단하고 효율적이지만 boundary 근처의 예측은 성능이 좋지 않다  
    - 2. 이미지를 resize하는 대신 inference speed를 높이기 위해 network의 채널을 prunning  
        - 이는 공간적 정보를 많이 잃는다는 문제가 발생한다  
    - 3. 네트워크의 마지막 stage를 drop  
        - 모델의 receptive field가 큰 object를 커버하기에 충분치 않으며 그 결과 성능이 좋지 않음  
          
          
- spatial detail의 손실을 교정하기 위해 U-shape 구조를 연구한 결과 2가지 문제점이 존재   
    - 1. 완전한 U-shape 구조는 연산량이 많아 inference 속도가 느리다  
    - 2. prunning or cropping에서 손실된 대부분의 spatial 정보가 shallow layer를 포함함으로써 복구가 불가능  
      
      
- 그래서 본 논문에서는 SP(Spatial Path), CP(Context Path)로 이루어진 BiseNet을 제안한다    
- Spatial Path는 spatial 정보의 손실을 막아주고   
- Context Path는 receptive field의 수축을 막아준다  
- SP에서는 3개의 Conv layer들을 쌓아 1/8 feature map들을 얻는다  
- CP에서는 xception 네트워크 마지막 부분에 GAP를 더해 receptive field 값을 backbone 네트워크의 최댓값으로 설정      


- inference 속도 감소 없이 더 좋은 정확도를 얻기 위해 SP, CP를 결합하고 FFM(Feature Fusion Module)와 ARM(Attention Refinement Module)을 제안한다    
    - 공간 정보 보존과 receptive field 제공을 두 개의 Path로 분리하기 위해 새롭게 접근  
    - FFM과 ARM 두 개의 새로운 모듈 제안하며 이는 성능을 개선시켜줄 것으로 기대  
    - city spaces, camvid 등 data set에서 68.4%의 정확도와 105FPS의 빠른 처리 속도를 가짐

## Bilateral Segmentation Network    
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiseNet/figure/figure2.JPG?raw=true">

### Spatial Path  
- 기존 semantic segmentation 모델들은 이미지의 spatial 정보들을 최대한 보존하려고 노력함  
- 즉, 이 task에서 높은 정확도를 달성하기 위해선 spatial 정보와 receptive field가 매우 중요함  
- 특히, real-time task에서 최근 모델들은 작은 input size나 속도를 올리기 위해 가벼운 base model을 활용함  
- 그러나 작은 input size는 spatial 정보를 대부분 잃으며 가벼운 모델은 feature channel이 충분하지 않음  
- 그래서 본 논문에서는 Spatial Path를 제안함  
- SP에서는 3개의 stride 2 Conv + Batch Normalization + ReLU layer로 구성되어 있음  
- SP의 output size는 input size의 1/8  
- 이는 feature map의 큰 spatial size 덕분에 풍부한 정보를 인코딩할 수 있다

### Context Path  
- SP에서 풍부한 spatial 정보를 인코딩하는 동안 CP에서는 충분한 receptive field를 제공함  
- receptive field를 확대하기 위해 일부 모델은 pyramid pooling module을 사용함 (ex. PSPNet)  
- 이는 계산 비용이 많이 들고 메모리도 많이 소모돼 속도가 느리다는 단점이 존재  
- 그래서 이를 고려한 Context Path를 제안  
- CP는 lightweight 모델과 large receptive field를 제공하기 위해 GAP를 활용함  
- Xception과 같은 lightweight 모델은 고수준의 semantic context 정보를 인코딩하며  
- large receptive field를 얻기 위해 feature map을 빠르게 down-sampling을 수행함  
- 그러면 global context 정보와 함께 maximum receptive field를 제공할 수 있은 lightweight 모델의 마지막에 GAP를 결합할 수 있음  
- 마지막으로 global pooling의 up-sampling된 output feature와 lightweight 모델의 feawture를 결합할 수 있음

### Attention Refinement Module  
- 본 논문은 CP의 각 stage에서 feature를 개선해주는 ARM이란 것을 제안한다  
- ARM은 global context를 캡쳐하기 위해 GAP를 이용하고 feature learning을 가이드하기 위해 attention 벡터를 계산함  
- 이는 CP에서 각 stage의 output feature를 개선시켜주며  
- up-sampling 연산 없이 global context 정보를 통합시킨다

### Feature Fusion Module  
- SP와 CP의 output은 서로 다른 feature 표현을 갖고 있음  
- 즉, SP의 feature는 low level이고 CP의 feature는 high level이다  
- 본 논문에서는 FFM이라는 것을 이용해 두 feature를 결합한다  
- 두 level feature가 주어지면 이 두 개를 이어붙인 후 BN을 활용해 feature scale을 맞춰준다   
- 그 다음으로 이어붙인 feature를 feature vector에 pooling을 하고 weight vector를 계산한다  
- 이 weight vector는 feature selection과 combination에 의해 다시 re-weight할 수 있음

##  Loss Function  
$$loss=\frac{1}{N}\sum_i-\log\left(\frac{e^{p_i}}{\sum_je^{p_j}}\right)$$  
$$L\left(X;W\right)=l_p\left(X;W\right)+\alpha\sum_{i=1}^Kl_i\left(X_i;W\right)$$

- $l_p$: principal loss  
- $l_i$: auxiliary loss  
- $X_i$: Xception model의 $i$번째 output  
- $K=3$

- auxiliary loss: BiseNet 훈련을 지도하기 위한 loss
- principal loss: BiseNet output을 지도하기 위한 loss  
- Softmax loss : BiseNet 전체의 loss  
- principal loss와 auxiliary loss의 weight를 조절하기 위해 $\alpha$가 존재  
- 논문에서는 $\alpha=1$을 사용

## Experimental Results  
- data set:  
    - Cityscapes  
    - CamVid  
    - COCO-Stuff  
    
### Implementation protocol  
- Network   
    - SP에는 3개의 conv layer, CP에는 Xception 39 model을 적용  
    - 그리고 FFM을 통해 SP와 CP 결합  
    
- Training details   
    - batch size: 16  
    - optimizer: SGD momentum 0.9  
    - weight decay: 1e-4  
    - learning rate: 2.5e-2  
    - scheduler: poly learning rate scheduler  
    
- Data augmentation  
    - mean subtraction, random horizontal flip, random scale  
    - scales: {0.75, 1.0, 1.5, 1.75, 2.0}  
    - 마지막으로 고정된 사이즈로 이미지를 crop

## Results   

<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiseNet/figure/figure3.JPG?raw=true">  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiseNet/figure/figure4.JPG?raw=true">
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiseNet/figure/table3.JPG?raw=true">  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiseNet/figure/table4.JPG?raw=true">
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiseNet/figure/table5.JPG?raw=true">  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiseNet/figure/table6.JPG?raw=true">
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiseNet/figure/table7.JPG?raw=true">  
<img src = "https://github.com/Sangh0/Segmentation/blob/main/BiseNet/figure/table8.JPG?raw=true">

## Conclusions  
- 본 논문에서 제안한 BiseNet에는 Spatial Path와 Context Path가 포함되어 있다  
- SP는 원본 이미지의 spatial 정보를 보존하도록 디자인되었고  
- CP는 lightweight 모델과 GAP를 